In [1]:
import os
from experiment_setup import setups
from model_tracker import track_training_C02_emissions

In [18]:
dataset = "retailrocket"    # coveo, diginetica, rees46, retailrocket, yoochoose
loss_function = "xe"  # Solo disponible en xe

dataset_path = f"../datasets/{dataset}"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run ../Preprocess/coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [21]:
params = setups[dataset][f"params_{loss_function}"]

In [22]:
train_path = os.path.join(dataset_path,f"{dataset}_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,f"{dataset}_processed_view_test.tsv")

In [23]:
def create_keras_gru4rec_script(model_name, train_path, test_path, model_path, loss, optim, layers, batch_size, dropout_p_hidden, learning_rate, n_epochs, m, use_correct_mask_reset):
    s_train_full = f"python ../KerasGRU4Rec/model/gru4rec.py --train_path {train_path} --test_path {test_path} --save_path {model_path}/{model_name} --lr {learning_rate} --hidden_size {layers} --dropout_p_hidden {dropout_p_hidden} --batch_size {batch_size}  --optim {optim} --epochs {n_epochs} {'--use_correct_mask_reset' if use_correct_mask_reset else ''}"
    s_test_full = f"python ../KerasGRU4Rec/model/gru4rec.py --train_path {train_path} --test_path {test_path} --resume {model_path}/KerasGRU4Rec/temp/GRU4REC_{n_epochs-1}.h5 --batch_size {batch_size} --eval_only True --m {m}"
    return s_train_full, s_test_full

In [26]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test (major fix model)

In [27]:
train_script_major, test_script_major = create_keras_gru4rec_script(model_name=f'keras_gru4rec_majorfix_{loss_function}', test_path=test_path, train_path=train_path, model_path=model_path, loss=loss, optim=optim, layers=layers, batch_size=batch_size, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, use_correct_mask_reset=True)

### Train

In [28]:
track_training_C02_emissions(train_script_major, f"keras_gru4rec_{loss_function}", dataset)

[codecarbon WARNING @ 00:25:48] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 00:25:48] [setup] RAM Tracking...
[codecarbon INFO @ 00:25:48] [setup] GPU Tracking...
[codecarbon INFO @ 00:25:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:25:50] [setup] CPU Tracking...
[codecarbon WARNING @ 00:25:50] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 00:25:51] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 00:25:51] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon WARNING @ 00:25:52] Failed to retrieve gpu information
Traceback (most recent call last):
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\codecarbon\core\gpu.py", line 238, in get_gpu_details
    devices_info.append(gpu_device.get_gpu_details())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

Salida de STDOUT: None
CO2_emissions_kg: 0.034205424769396475


0.034205424769396475

### Test

In [29]:
print(test_script_major)
# os.system(test_script_major)

python ../KerasGRU4Rec/model/gru4rec.py --train_path ../datasets/rees46\rees46_processed_view_train_full.tsv --test_path ../datasets/rees46\rees46_processed_view_test.tsv --resume ../trained_models/KerasGRU4Rec/temp/GRU4REC_4.h5 --batch_size 240 --eval_only True --m 1 5 10 20
